In [1]:
import dendropy.calculate.treecompare
import MakingSequences
import makingPairwiseDistancesOurMethod
import makingFinalReconstructedTreeOurMethod
import makingOtherMethodTree
import comparingTrees
import time

In [2]:
seqLenInput = 1e7
initial_size_Var=100
# time_Var = 200_000
mu_Var = 1e-8
recombination_rate=1e-8

In [3]:
birth_rate = 1000.0
death_rate = 500.0
n_populations = 5
extra_populations = 5 # for "GSA" sampling scheme

In [4]:
st = time.time()
seqs, tree = MakingSequences.makeSequencesFromScratch(birth_rate, death_rate, n_populations, extra_populations, seqLenInput, initial_size_Var, mu_Var, recombination_rate)
print((time.time()-st)/60)

Demography
╟  Populations
║  ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
║  │ id │name   │description  │initial_size  │ growth_rate │  default_sampling_time│extra_metadata  │
║  ├────────────────────────────────────────────────────────────────────────────────────────────────┤
║  │ 0  │T5     │             │100.0         │      0      │                      0│{}              │
║  │ 1  │T4     │             │100.0         │      0      │                      0│{}              │
║  │ 2  │T3     │             │100.0         │      0      │                      0│{}              │
║  │ 3  │T2     │             │100.0         │      0      │                      0│{}              │
║  │ 4  │T1     │             │100.0         │      0      │                      0│{}              │
║  │ 5  │pop_5  │             │100.0         │      0      │                2.1e+04│{}              │
║  │ 6  │pop_6  │             │100.0         │      0   

In [5]:
st = time.time()
mats = makingFinalReconstructedTreeOurMethod.makeManyMatricesOfDistances(10, seqs)
print((time.time()-st)/60)

in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0
in getting kmer vectors, working on k: 12 seq 1
in getting kmer vectors, working on k: 12 seq 2
in getting kmer vectors, working on k: 12 seq 3
in getting kmer vectors, working on k: 12 seq 4
working on inferring 0 1
working on inferring

/Users/victorkaplan/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 1 3
working on inferring 1 4
working on inferring 2 3
working on inferring 2 4
working on inferring 3 4
in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0
in getting kmer vectors, working on k: 12 seq 1
in getting kmer vectors, working on k: 12 seq 2
in getting kmer 

/Users/victorkaplan/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 1 3
working on inferring 1 4
working on inferring 2 3
working on inferring 2 4
working on inferring 3 4
in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0
in getting kmer vectors, working on k: 12 seq 1
in getting kmer vectors, working on k: 12 seq 2
in getting kmer 

...

working on inferring 1 3
working on inferring 1 4
working on inferring 2 3
working on inferring 2 4
working on inferring 3 4
in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0
in getting kmer vectors, working on k: 12 seq 1
in getting kmer vectors, working on k: 12 seq 2
in getting kmer 

/Users/victorkaplan/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 1 3
working on inferring 1 4
working on inferring 2 3
working on inferring 2 4
working on inferring 3 4
in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0
in getting kmer vectors, working on k: 12 seq 1
in getting kmer vectors, working on k: 12 seq 2
in getting kmer 

/Users/victorkaplan/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 1 3
working on inferring 1 4
working on inferring 2 3
working on inferring 2 4
working on inferring 3 4
32.56233658393224


In [36]:
finalDistMat = makingFinalReconstructedTreeOurMethod.makeFinalDistMatrixFromSmallerOnes(mats)

[[10.  2.  1.  7.  4.]
 [ 2. 10.  1.  4.  4.]
 [ 1.  1. 10.  6.  1.]
 [ 7.  4.  6. 10.  5.]
 [ 4.  4.  1.  5. 10.]]


In [37]:
import numpy as np
print(finalDistMat)
average = np.average(finalDistMat)
for i in range(len(finalDistMat[0])):
    for j in range(len(finalDistMat[0])):
        if i != j:
            if finalDistMat[i][j] == 0.0:
                finalDistMat[i][j] += average
finalDistMat

          0         1         2         3         4
0  0.000000  0.000239  0.000241  0.000395  0.000270
1  0.000239  0.000000  0.000186  0.000358  0.000328
2  0.000241  0.000186  0.000000  0.000341  0.000227
3  0.000395  0.000358  0.000341  0.000000  0.000255
4  0.000270  0.000328  0.000227  0.000255  0.000000


,0,1,2,3,4
0,0.000000,0.000239,0.000241,0.000395,0.000270
1,0.000239,0.000000,0.000186,0.000358,0.000328
2,0.000241,0.000186,0.000000,0.000341,0.000227
3,0.000395,0.000358,0.000341,0.000000,0.000255
4,0.000270,0.000328,0.000227,0.000255,0.000000


In [76]:
# our_tree = makingFinalReconstructedTreeOurMethod.makeTree(seqs, mu_Var , (finalDistMat*2))


# print(our_tree.as_string("newick"))
# our_tree.print_plot()


In [39]:
theirTree = makingOtherMethodTree.inferTreeFromSeqs(seqs, mu_Var)

T5	0
T4	0.0007721000000000533	0
T3	0.0007654000000000272	0.0004311000000000176	0
T2	0.000825100000000023	0.0008407999999999749	0.000833900000000054	0
T1	0.0008137000000000283	0.0008295000000000385	0.0008224999999999483	0.0007513000000000103	0
	T5	T4	T3	T2	T1


In [104]:
theirTreeMatrix = np.array(pd.read_csv('otherMethodDistanceMatrix.csv').drop(columns='Unnamed: 0'))

In [102]:
trueDistDict = tree.phylogenetic_distance_matrix().as_data_table()._data
trueDistMat = np.zeros((5,5))
for pair1 in trueDistDict:
    for pair2 in trueDistDict:
        ind1 = list(seqs.keys()).index(pair1)
        ind2 = list(seqs.keys()).index(pair2)  
        trueDistMat[ind1][ind2]  = trueDistDict[pair1][pair2]

import pandas as pd      
print(pd.DataFrame(trueDistMat*mu_Var))

          0         1         2         3         4
0  0.000000  0.000762  0.000762  0.000830  0.000830
1  0.000762  0.000000  0.000425  0.000830  0.000830
2  0.000762  0.000425  0.000000  0.000830  0.000830
3  0.000830  0.000830  0.000830  0.000000  0.000757
4  0.000830  0.000830  0.000830  0.000757  0.000000


In [78]:
np.linalg.norm(finalDistMat*2 - trueDistMat*mu_Var)

0.0010148698009430762

In [105]:
np.linalg.norm(theirTreeMatrix - trueDistMat*mu_Var)

3.638292347136394e-05

In [25]:
# print('==== REAL')
# pdc = tree.phylogenetic_distance_matrix()
# for i, t1 in enumerate(tree.taxon_namespace[:-1]):
#     for t2 in tree.taxon_namespace[i+1:]:
#         print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
# print('====')
# print('==== THEM')
# pdc = theirTree.phylogenetic_distance_matrix()
# for i, t1 in enumerate(theirTree.taxon_namespace[:-1]):
#     for t2 in theirTree.taxon_namespace[i+1:]:
#         print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
# print('====')
# print('==== US')
# pdc = our_tree.phylogenetic_distance_matrix()
# for i, t1 in enumerate(our_tree.taxon_namespace[:-1]):
#     for t2 in our_tree.taxon_namespace[i+1:]:
#         print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
# print('====')

In [8]:
# from dendropy import Tree

In [9]:

# for edge in our_tree_2.postorder_edge_iter():
#     if edge.length != None:
#         edge.length *= 100

In [10]:
# comparingTrees.computeDistancesOfTrees(theirTree, tree, n_populations)

In [11]:
# comparingTrees.computeDistancesOfTrees(tree, our_tree, n_populations)

In [12]:
# print(dendropy.calculate.treecompare.robinson_foulds_distance(theirTree_2, tree))
# print(dendropy.calculate.treecompare.robinson_foulds_distance(our_tree_2, tree))
# 

In [13]:
# tree.taxon_namespace

In [14]:
# our_tree.taxon_namespace

In [15]:
# tns = tree.taxon_namespace

In [16]:
# our_tree.taxon_namespace = tns

In [17]:
# our_tree.taxon_namespace
# 

In [18]:
# tree.taxon_namespace

In [19]:
# our_tree_2 = Tree(our_tree, taxon_namespace=tns)

In [20]:
# print(str(our_tree_2))

In [21]:
# theirTree_2 = Tree(theirTree, taxon_namespace=tns)

In [22]:
# theirTree.print_plot()
# theirTree_2.print_plot()
# print(theirTree.as_string('newick'))
# print(theirTree_2.as_string('newick'))

# ((T2:280025.0000000018,T3:37135.00000000341):6029.999999999231,(T1:24814.999999998865,(T4:8975.000000001804,T5:8824.999999999343):15869.999999996997):6029.999999999231)

In [23]:
# theirTree.print_plot()

In [24]:
# print(theirTree is not theirTree_2)